In [32]:
#imports
from __future__ import print_function
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from mpl_toolkits.mplot3d import Axes3D
from pyspark.sql.functions import col
from functools import reduce
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.functions import col, split, size, isnan, array_contains, array_min, when, count
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, FloatType
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import pandas as pd 
import glob
import re
import pathlib
import json
import datetime
import numpy as np
import time
import os

In [2]:
#paths
DATASET_PATH='/Users/Soroush/Desktop/Thesis/Code/dataset/'

#from google.colab import drive
#drive.mount('/gdrive')

# **Install spark**

In [ ]:
# Install spark and dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.its.dal.ca/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz #or any other version
!tar xvf spark-2.4.5-bin-hadoop2.7.tgz #based on version
!pip install -q findspark

# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7" #based on version

#imports
import findspark
findspark.init()
import pyspark # Call this only after findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

#test
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
file_loc = './sample_data/california_housing_train.csv'
df = spark.read.csv(file_loc, inferSchema=True, header =True)
df.show()

# **Load Dataset**

In [3]:
#load and save .read_pickle() and .to_pickle() 

#save

#dataset.to_pickle(DATASET_PATH+"dataset.pkl")
#aggregated_dataset.to_pickle(DATASET_PATH+"aggregated_dataset.pkl")
#json_dataset.to_pickle(DATASET_PATH+"json_dataset.pkl")
#dataset.to_csv(DATASET_PATH+"dataset.csv")
#aggregated_dataset.to_csv(DATASET_PATH+"aggregated_dataset.csv")
#json_dataset.to_csv(DATASET_PATH+"json_dataset.csv")
#aggregated_dataset_rowBased.to_csv(DATASET_PATH+"aggregated_dataset_rowBased.csv")


#load

dataset=pd.read_pickle(DATASET_PATH+"dataset.pkl")
aggregated_dataset=pd.read_pickle(DATASET_PATH+"aggregated_dataset.pkl")
json_dataset=pd.read_pickle(DATASET_PATH+"json_dataset.pkl")

In [ ]:
#load dataset

# dataset address: http://traces.cs.umass.edu/index.php/Smart/Smart

#Extract File
#import tarfile
#!tar -xf '/gdrive/My Drive/a.gzip' -C '/gdrive/My Drive/'

#load

def load_smart_star_dataset(dataset_path): 
    
    #length of file path
    LENGTH=len(dataset_path)+5

    df_merged = pd.DataFrame(columns=['date'])
    
    # 2014
    #path_2014 = r'/gdrive/My Drive/Dataset/apartment/2014'
    path_2014= dataset_path+'2014'
    all_2014_paths = glob.glob(path_2014 + "/Apt*.csv")
    df_merged_2014 = pd.DataFrame(columns=['date'])
    for file_name in all_2014_paths:
        column_name = file_name[LENGTH:-9]
        # column_name = file_name.replace("dataset/2014/", "").replace("_2014.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2014 = pd.merge(df_merged_2014, df, on='date', how='outer', left_index=True, right_index=True)
    
    # 2015
    df_merged_2015 = pd.DataFrame(columns=['date'])
    #path_2015 = r'/gdrive/My Drive/Dataset/apartment/2015'
    path_2015= dataset_path+'2015'
    all_2015_paths = glob.glob(path_2015 + "/Apt*.csv")
    for file_name in all_2015_paths:
        column_name = file_name[LENGTH:-9]
        # column_name = file_name.replace("dataset/2015/", "").replace("_2015.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2015 = pd.merge(df_merged_2015, df, on='date', how='outer', left_index=True, right_index=True)
    
    # 2016
    df_merged_2016 = pd.DataFrame(columns=['date'])
    #path_2016 = r'/gdrive/My Drive/Dataset/apartment/2016'
    path_2016= dataset_path+'2016'
    all_2016_paths = glob.glob(path_2016 + "/Apt*.csv")
    for file_name in all_2016_paths:
        column_name = file_name[LENGTH:-9]
        # column_name = file_name.replace("dataset/2016/", "").replace("_2016.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2016 = pd.merge(df_merged_2016, df, on='date', how='outer', left_index=True, right_index=True)
    
    # merge all years
    df_merged = df_merged_2014.append(df_merged_2015, ignore_index=True).append(df_merged_2016, ignore_index=True)
    final = df_merged
    #save
    return final

# **Preprocessing**

In [ ]:
#aggregate data

def agg_by_date(final):
    final['date'] = pd.to_datetime(final['date'],)
    final.index=final['date']

    final_agg_by_hour = final.resample('60T').sum(min_count=1)  #for more than to NaN : .apply(lambda x: x.sum() if x.isnull().sum() <= 2 else np.nan)

    final = final_agg_by_hour.resample('D').aggregate(lambda x: x.tolist()) # for tuple: .aggregate(lambda x: tuple(x))

    #remove first and last row
    #aggregated_dataset.drop(pd.to_datetime('2016-12-28'),inplace=True)
    #aggregated_dataset.drop(pd.to_datetime('2014-10-15'),inplace=True)
    #final=final.iloc[1:-1]

    return final

In [ ]:
#create json data from stored dataframe

def to_json(final):
    PERIOD=60

    data_for_json=final.loc[:, final.columns != 'date']

    def date_to_str(o):
        if isinstance(o, datetime.datetime):
            return o.__str__()

    json_dataframe = pd.DataFrame(columns=['data']) #creates a new dataframe that's empty
    L  = []

    import json
    import datetime
    import time

    r, c = data_for_json.shape
    for i in range(0, r):
        for j in range(0, c):
            data = {}
            data['id'] = data_for_json.columns.values[j]
            data['power'] = data_for_json.iloc[i][j]
            data['date']=data_for_json.index.tolist()[i]
            json_data = json.dumps(data,default=date_to_str)
            L.append(json_data)
            #json_dataframe=json_dataframe.append(json_data,ignore_index=True)

    json_dataframe = pd.DataFrame(L, columns=['data'])
    return json_dataframe

In [ ]:
#convert column_based df to row_based

def col_to_row(df):
    #house_id=df.columns
    df.reset_index(inplace=True)
    return pd.melt(df, id_vars=['date'],value_name='power')

#aggregated_dataset=pd.read_pickle(DATASET_PATH+"aggregated_dataset.pkl")
#aggregated_dataset_rowBased=col_to_row(aggregated_dataset)
#aggregated_dataset_rowBased.sort_values(['date', 'variable'], ascending=[True, True],inplace=True)

# **Model**

In [97]:
#create SparkSession
spark = SparkSession.builder.appName("anomaly_detection").master("local[2]").getOrCreate()

#define schema
schema = StructType([
    StructField("#", IntegerType()),
    StructField("date", TimestampType()),
    StructField("id", StringType()),
    StructField("power", StringType())])

#read data
sdf = spark.read.format('csv').options(header='true',inferSchema=True ,schema=schema).load(DATASET_PATH+"aggregated_dataset_rowBased.csv")

#rename columns
def rename_dataframe(sdf):
    names = ['#','date','id','power']
    for c,n in zip(sdf.columns,names):
        sdf=sdf.withColumnRenamed(c,n)
    return sdf
    
sdf=rename_dataframe(sdf)
    
#convert power to array
def string_power_to_array(sdf):
    temp=sdf.withColumn("power", f.regexp_replace(f.regexp_replace(f.col("power"), "\\[", ""),"\\]","")
                        .alias("power"))
    temp=temp.withColumn("power",split(col("power"), ",\s*")
                        .cast(ArrayType(FloatType())).alias("power"))
    return temp

sdf=string_power_to_array(sdf)
sdf.show()

#add validation column
def add_validation_column(sdf):
    v=True
    temp=sdf.withColumn("V",f.when((size(sdf.power)==24) #array size
                                   & ~(f.array_contains(sdf.power, float('nan'))) #containing NaN
                                   & ~(f.array_max(sdf.power)<=0) #all 0
                                   , True).otherwise(False))
    return temp

sdf=add_validation_column(sdf)
sdf.show(120)

def split_power(sdf):
    temp=sdf.select("#","date","id",
           sdf.power[0].alias("H0"), sdf.power[1].alias("H1"), sdf.power[2].alias("H2"),sdf.power[3].alias("H3"),
           sdf.power[4].alias("H4"), sdf.power[5].alias("H5"), sdf.power[6].alias("H6"),sdf.power[7].alias("H7"),
           sdf.power[8].alias("H8"), sdf.power[9].alias("H9"), sdf.power[10].alias("H10"),sdf.power[11].alias("H11"),
           sdf.power[12].alias("H12"), sdf.power[13].alias("H13"), sdf.power[14].alias("H14"),sdf.power[15].alias("H15"),
           sdf.power[16].alias("H16"), sdf.power[17].alias("H17"), sdf.power[18].alias("H18"),sdf.power[19].alias("H19"),
           sdf.power[20].alias("H20"), sdf.power[21].alias("H21"), sdf.power[22].alias("H22"),sdf.power[23].alias("H23"))
    return temp
    
#sdf=split_power(sdf)
#sdf.show()





#sdf.printSchema()

+---+-------------------+-----+--------------------+
|  #|               date|   id|               power|
+---+-------------------+-----+--------------------+
|  0|2014-08-15 00:00:00|Apt36|[1.96932, 0.38400...|
|  1|2014-08-16 00:00:00|Apt36|[0.61410224, 0.44...|
|  2|2014-08-17 00:00:00|Apt36|[1.3780422, 1.165...|
|  3|2014-08-18 00:00:00|Apt36|[1.1634789, 2.013...|
|  4|2014-08-19 00:00:00|Apt36|[1.1126077, 1.294...|
|  5|2014-08-20 00:00:00|Apt36|[1.4072489, 1.076...|
|  6|2014-08-21 00:00:00|Apt36|[1.4892989, 1.260...|
|  7|2014-08-22 00:00:00|Apt36|[0.5883856, 0.554...|
|  8|2014-08-23 00:00:00|Apt36|[0.52373445, 1.48...|
|  9|2014-08-24 00:00:00|Apt36|[0.39280555, 1.30...|
| 10|2014-08-25 00:00:00|Apt36|[0.4686211, 0.502...|
| 11|2014-08-26 00:00:00|Apt36|[1.2439622, 0.637...|
| 12|2014-08-27 00:00:00|Apt36|[1.8882623, 2.731...|
| 13|2014-08-28 00:00:00|Apt36|[1.29602, 1.83715...|
| 14|2014-08-29 00:00:00|Apt36|[1.3385545, 1.459...|
| 15|2014-08-30 00:00:00|Apt36|[1.7708489, 3.3

# **MAIN**

In [ ]:
#load dataset
dataset = load_smart_star_dataset(DATASET_PATH)

#aggregate by date
aggregated_dataset=agg_by_date(dataset)
aggregated_dataset=aggregated_dataset[1:-1]

#to json
json_dataset=to_json(aggregated_dataset)

# **TEST**

In [ ]:
#sdf.printSchema()

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

#cast
def string_to_list(t: str):
    return t.strip('][').split(', ') 
     

sdf.withColumn("greetings", string_to_list(col("power"))).show()

In [ ]:
from pyspark.sql.types import *

schema = StructType([
  StructField("num", IntegerType()),
    StructField("date", TimestampType()),
    StructField("id", StringType()),
  StructField("power", ArrayType(
      StructType([
          StructField("H0", FloatType(), True),
          StructField("H1", FloatType(), True),
          StructField("H2", FloatType(), True),
          StructField("H3", FloatType(), True),
          StructField("H4", FloatType(), True),
          StructField("H5", FloatType(), True),
          StructField("H6", FloatType(), True),
          StructField("H7", FloatType(), True),
          StructField("H8", FloatType(), True),
          StructField("H9", FloatType(), True),
          StructField("H10", FloatType(), True),
          StructField("H11", FloatType(), True),
          StructField("H12", FloatType(), True),
          StructField("H13", FloatType(), True),
          StructField("H14", FloatType(), True),
          StructField("H15", FloatType(), True),
          StructField("H16", FloatType(), True),
          StructField("H17", FloatType(), True),
          StructField("H18", FloatType(), True),
          StructField("H19", FloatType(), True),
          StructField("H20", FloatType(), True),
          StructField("H21", FloatType(), True),
          StructField("H22", FloatType(), True),
          StructField("H23", FloatType(), True)
      ])
   )
             )])

a = spark.read.format('csv').schema(schema).option("header", "true").load(DATASET_PATH+"f.csv")
a.show()

In [ ]:
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)

spark = SparkSession.builder.appName("anomaly_detection").master("local[2]").getOrCreate()

#load dataset
sdf = spark.read.format('csv').options(header='true', inferSchema='true').load(DATASET_PATH+"aggregated_dataset_rowBased.csv")

#cast
def string_to_list(t):
    return t.strip('][').split(', ') 

sdf.show()
#sdf.take(10)

In [14]:

#c = sdf.withColumn('new_column',func_udf(sdf['_c0']))
#c=sdf.withColumn("V",f.when((size(sdf.power)==24) , True).otherwise(False))

a = spark.read.format('csv').schema(schema).option("header", "true").load(DATASET_PATH+"f.csv")

def string_power_to_array(sdf):
    temp=sdf.withColumn("power", f.regexp_replace(f.regexp_replace(f.col("power"), "\\[", ""),"\\]","")
                        .alias("power"))
    temp=temp.withColumn("power",split(col("power"), ",\s*")
                        .cast(ArrayType(FloatType())).alias("power"))
    return temp

a=string_power_to_array(a)

a.printSchema()

a=a.withColumn("V1",f.when((size(a.power)==24) , True).otherwise(False))
#a=a.withColumn("V2",isnan(a.power))
#a=a.withColumn("V3",col("power").isNull())
#a=a.withColumn("V4",a.power> 0)
#a=a.withColumn("V5",array_contains(col("power"), 1.0))
a.show()

b=a.select("num","date","id","power")
b.show()


c=a.select("num","date","id",
           a.power[0].alias("H0"), a.power[1].alias("H1"), a.power[2].alias("H2"),a.power[3].alias("H3"),
           a.power[4].alias("H4"), a.power[5].alias("H5"), a.power[6].alias("H6"),a.power[7].alias("H7"),
           a.power[8].alias("H8"), a.power[9].alias("H9"), a.power[10].alias("H10"),a.power[11].alias("H11"),
           a.power[12].alias("H12"), a.power[13].alias("H13"), a.power[14].alias("H14"),a.power[15].alias("H15"),
           a.power[16].alias("H16"), a.power[17].alias("H17"), a.power[18].alias("H18"),a.power[19].alias("H19"),
           a.power[20].alias("H20"), a.power[21].alias("H21"), a.power[22].alias("H22"),a.power[23].alias("H23")
          )
c.show()
c.printSchema()



root
 |-- num: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- id: string (nullable = true)
 |-- power: array (nullable = true)
 |    |-- element: float (containsNull = true)

+---+-------------------+-----+--------------------+----+
|num|               date|   id|               power|  V1|
+---+-------------------+-----+--------------------+----+
|  0|2014-08-15 00:00:00|Apt36|[1.96932, 0.38400...|true|
|  1|2014-08-16 00:00:00|Apt36|[0.61410224,, 0.3...|true|
+---+-------------------+-----+--------------------+----+

+---+-------------------+-----+--------------------+
|num|               date|   id|               power|
+---+-------------------+-----+--------------------+
|  0|2014-08-15 00:00:00|Apt36|[1.96932, 0.38400...|
|  1|2014-08-16 00:00:00|Apt36|[0.61410224,, 0.3...|
+---+-------------------+-----+--------------------+

+---+-------------------+-----+----------+----------+----------+----------+---------+-----------+----------+-----------+----------+---